In [1]:
import os
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage
from IPython.display import display, Markdown, Latex
from datetime import datetime

In [2]:
API_KEY = os.environ['MISTRAL_API_KEY']

MODEL_ID = 'open-mixtral-8x22b'
MODEL_SEED = 2
MODEL_TEMPERATURE = 0.7

file_system_prompt_1 = open("../../system_prompt_1.txt", "r")
file_system_prompt_2 = open("../../system_prompt_2.txt", "r")
SYSTEM_PROMPT_1 = file_system_prompt_1.read()
SYSTEM_PROMPT_2 = file_system_prompt_2.read()
file_system_prompt_1.close()
file_system_prompt_2.close()

FILE_PATH_PROBLEM = '../../../../Datasets/IP_2_Choosing_Investment_Strategies/'
assert(FILE_PATH_PROBLEM != '../../../../Datasets/')
file_problem_description = open(FILE_PATH_PROBLEM + 'ProblemDescription.txt', 'r')
PROBLEM_DESCRIPTION = file_problem_description.read()
file_problem_description.close()


client = MistralClient(api_key=API_KEY)

print(f'Time of execution: {datetime.now()}')

Time of execution: 2024-05-18 13:36:13.900636


## Step 1 - Generate Mathematical Formulation 

In [3]:
messages_1 = [
    ChatMessage(role="system", content=SYSTEM_PROMPT_1),
    ChatMessage(role="user", content=PROBLEM_DESCRIPTION)
]

In [4]:
response_1 = client.chat(
    model=MODEL_ID,
    messages=messages_1,
    random_seed=MODEL_SEED,
    temperature=MODEL_TEMPERATURE 
)

response_1_text = response_1.choices[0].message.content

In [5]:
Markdown(response_1_text)

To formulate a mean-variance mathematical model for this optimization problem, we will use the following parameters, decision variables, objective function, and constraints:

Parameters:
- \(N\) is the number of investment strategies (10 in this case).
- \(A\) is the cost vector, where \(A_i\) is the cost of investing in strategy \(i\).
- \(R\) is the return matrix, where \(R_{ij}\) is the return of strategy \(i\) in scenario \(j\).
- \(B\) is the budget scenario vector, where \(B_j\) is the budget for scenario \(j\).
- \(P\) is the probability vector, where \(P_j\) is the probability of budget scenario \(j\).

Decision Variables:
- \(x_i \in \{0,1\}\) is a binary decision variable that represents whether or not to invest in strategy \(i\).

Objective Function:
Maximize the expected return with a risk parameter \(r\) (set to 2 in this case):
\[ \max \left( \sum_{i=1}^{N} \left( \frac{1}{M} \sum_{j=1}^{M} R_{ij} \right) x_i - r \sum_{i=1}^{N} \left( \frac{1}{M} \sum_{j=1}^{M} (R_{ij} - \mu)^2 \right) x_i \right) \]

where \(M\) is the number of budget scenarios, and \(\mu\) is the mean return of all strategies in all scenarios:
\[ \mu = \frac{1}{N \cdot M} \sum_{i=1}^{N} \sum_{j=1}^{M} R_{ij} \]

Constraints:
1. Budget constraint: The total cost of investments should not exceed the budget with a tolerable probability of exceeding the budget of 0.4.
\[ \sum_{i=1}^{N} A_i x_i \leq \sum_{j=1}^{M} B_j P_j \cdot (1 - 0.4) \]

2. Investment constraint: You can only invest once into an investment.
\[ \sum_{i=1}^{N} x_i \leq 1 \]

This mean-variance optimization model aims to maximize the expected return while minimizing the risk, considering the past performance of investment strategies and the uncertain budget. The decision variables \(x_i\) indicate whether to invest in each strategy, and the constraints ensure that the budget is not exceeded and that each strategy is invested in at most once.

In [6]:
print(response_1_text)

To formulate a mean-variance mathematical model for this optimization problem, we will use the following parameters, decision variables, objective function, and constraints:

Parameters:
- \(N\) is the number of investment strategies (10 in this case).
- \(A\) is the cost vector, where \(A_i\) is the cost of investing in strategy \(i\).
- \(R\) is the return matrix, where \(R_{ij}\) is the return of strategy \(i\) in scenario \(j\).
- \(B\) is the budget scenario vector, where \(B_j\) is the budget for scenario \(j\).
- \(P\) is the probability vector, where \(P_j\) is the probability of budget scenario \(j\).

Decision Variables:
- \(x_i \in \{0,1\}\) is a binary decision variable that represents whether or not to invest in strategy \(i\).

Objective Function:
Maximize the expected return with a risk parameter \(r\) (set to 2 in this case):
\[ \max \left( \sum_{i=1}^{N} \left( \frac{1}{M} \sum_{j=1}^{M} R_{ij} \right) x_i - r \sum_{i=1}^{N} \left( \frac{1}{M} \sum_{j=1}^{M} (R_{ij} - 

## Step 2 - Generate the Pyomo Code

In [7]:
messages_2 = [
    ChatMessage(role="system", content=SYSTEM_PROMPT_2),
    ChatMessage(role="user", content=response_1_text)
]

In [8]:
response_2 = client.chat(
    model=MODEL_ID,
    messages=messages_2,
    random_seed=MODEL_SEED,
    temperature=MODEL_TEMPERATURE
)

response_2_text = response_2.choices[0].message.content

In [9]:
Markdown(response_2_text)

Here is a Python Pyomo code for the given optimization problem. I have used sample data for the parameters as indicated in the comments.

```python
import pyomo.environ as pyo
import numpy as np

# Sample data
N = 10  # number of investment strategies
M = 5  # number of budget scenarios
A = np.random.rand(N)  # cost vector
R = np.random.rand(N, M)  # return matrix
B = np.random.rand(M)  # budget scenario vector
P = np.random.rand(M)  # probability vector
P = P / np.sum(P)  # normalize probability vector

r = 2  # risk parameter

# Pyomo model
model = pyo.ConcreteModel()

# Decision variables
model.x = pyo.Var(range(N), domain=pyo.Binary)

# Objective function
def obj_rule(model):
    mu = np.mean(R)  # mean return of all strategies in all scenarios
    return (
        np.sum([np.mean(R[:, j]) * model.x[i] for i, j in np.ndindex(R.shape)])
        - r * np.sum([np.mean((R[:, j] - mu) ** 2) * model.x[i] for i, j in np.ndindex(R.shape)])
    )

model.obj = pyo.Objective(rule=obj_rule, sense=pyo.maximize)

# Constraints
def budget_constraint_rule(model):
    return np.dot(A, model.x) <= np.dot(B, P) * (1 - 0.4)

model.budget_constraint = pyo.Constraint(rule=budget_constraint_rule)

def investment_constraint_rule(model):
    return np.sum(model.x) <= 1

model.investment_constraint = pyo.Constraint(rule=investment_constraint_rule)

# Solve the model
solver = pyo.SolverFactory('glpk')
results = solver.solve(model)

# Print the results
print("Objective value:", pyo.value(model.obj))
print("Decision variables:", {i: pyo.value(model.x[i]) for i in range(N)})
```

This code defines the Pyomo model with the given objective function and constraints. It also generates sample data for the parameters and solves the model using the GLPK solver. The results include the objective value and the values of the decision variables.

In [10]:
print(response_2_text)

Here is a Python Pyomo code for the given optimization problem. I have used sample data for the parameters as indicated in the comments.

```python
import pyomo.environ as pyo
import numpy as np

# Sample data
N = 10  # number of investment strategies
M = 5  # number of budget scenarios
A = np.random.rand(N)  # cost vector
R = np.random.rand(N, M)  # return matrix
B = np.random.rand(M)  # budget scenario vector
P = np.random.rand(M)  # probability vector
P = P / np.sum(P)  # normalize probability vector

r = 2  # risk parameter

# Pyomo model
model = pyo.ConcreteModel()

# Decision variables
model.x = pyo.Var(range(N), domain=pyo.Binary)

# Objective function
def obj_rule(model):
    mu = np.mean(R)  # mean return of all strategies in all scenarios
    return (
        np.sum([np.mean(R[:, j]) * model.x[i] for i, j in np.ndindex(R.shape)])
        - r * np.sum([np.mean((R[:, j] - mu) ** 2) * model.x[i] for i, j in np.ndindex(R.shape)])
    )

model.obj = pyo.Objective(rule=obj_rule, se

### Code Executability

In [11]:
import pyomo.environ as pyo
import numpy as np

# Sample data
N = 10  # number of investment strategies
M = 5  # number of budget scenarios
A = np.random.rand(N)  # cost vector
R = np.random.rand(N, M)  # return matrix
B = np.random.rand(M)  # budget scenario vector
P = np.random.rand(M)  # probability vector
P = P / np.sum(P)  # normalize probability vector

r = 2  # risk parameter

# Pyomo model
model = pyo.ConcreteModel()

# Decision variables
model.x = pyo.Var(range(N), domain=pyo.Binary)

# Objective function
def obj_rule(model):
    mu = np.mean(R)  # mean return of all strategies in all scenarios
    return (
        np.sum([np.mean(R[:, j]) * model.x[i] for i, j in np.ndindex(R.shape)])
        - r * np.sum([np.mean((R[:, j] - mu) ** 2) * model.x[i] for i, j in np.ndindex(R.shape)])
    )

model.obj = pyo.Objective(rule=obj_rule, sense=pyo.maximize)

# Constraints
def budget_constraint_rule(model):
    return np.dot(A, model.x) <= np.dot(B, P) * (1 - 0.4)

model.budget_constraint = pyo.Constraint(rule=budget_constraint_rule)

def investment_constraint_rule(model):
    return np.sum(model.x) <= 1

model.investment_constraint = pyo.Constraint(rule=investment_constraint_rule)

# Solve the model
solver = pyo.SolverFactory('glpk')
results = solver.solve(model)

# Print the results
print("Objective value:", pyo.value(model.obj))
print("Decision variables:", {i: pyo.value(model.x[i]) for i in range(N)})

2024-05-18 13:43:42,927 ERROR pyomo.core: Rule failed when generating expression for Constraint investment_constraint with index None:
UnboundLocalError: local variable 'args' referenced before assignment
2024-05-18 13:43:42,928 ERROR pyomo.core: Constructing component 'investment_constraint' from data=None failed:
UnboundLocalError: local variable 'args' referenced before assignment


UnboundLocalError: local variable 'args' referenced before assignment

### Solution Correctness